<h1 style="color:#555; font-size:3em">Generador de fake data para la base de datos 'Laboratorio'</h1>
<h2 style="color:#555">Curso de SQL de CoderHouse</h2>

AUTOR: LEANDRO HORNOS

Este notebook contiene el código para generar data ficticia, con el fin de rellenar de información la base de datos MySQL del proyecto final del curso de SQL de CoderHouse, 2022. <br>
Para generar los datos se utiliza la librería Faker, la cual cuenta con la capacidad de generar un gran abanico de datos inventados en distintos idiomas. Dado que algunas funciones no están disponibles en español (por ejemplo el lorem), se utiliza el traductor de Google mediante la librería Deep Translator para pasar del inglés al español. Tener en cuenta que se requiere una conexión a internet para ejecutar el código por la necesidad de usar el servicio de traducción de Google. <br>
Los scripts pueden ser lentos dependiendo de la velocidad de conexión a internet y la cantidad de texto a traducir.

[Tabla 16: Movimiento Equipos](#tabla-16)

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
import random

from faker import Faker
from faker.providers import person

from deep_translator import GoogleTranslator

In [2]:
# set the seed
Faker.seed(100)

# Uso el locale en español de España
fake = Faker('es_ES')

# El locale solo tiene algunas cosas, por ejemplo el lorem queda en latin
# para los casos en que necesito explícitamente indicar que use faker en inglés
# genero una segunda instancia de faker en ingles.
fake_en = Faker('en_US')

In [3]:
# Encierro la lógica de la generación de cada tabla
# en una función que devuelve un dataframe.
# El template de la función es:

def generateTableDf_TABLENAME(count=100):
    data = {}
    
    df = pd.DataFrame(data)
    
    return df

**Funciones generales**

In [145]:
def removeAccentedChars(text):
    chars = [('á','a'),('é','e'), ('í','i'), ('ó','o'), ('ú','u'),
             ('Á','A'),('É','E'),('Í','I'),('Ó','O'),('Ú','U')]
    
    for char in chars:
        text = text.replace( char[0], char[1])
    return text

def toSpanish(text):
    try:
        translated = GoogleTranslator(source='auto', target='es').translate(text)
    except Exception as e:
        print(e)
        return text
    
    clean = removeAccentedChars(translated)
    return clean

def generateParentChildPairs(parentList, childrenList, offspring=2):
    
    # Eliminamos los duplicados
    
    if len( parentList ) != len( set(parentList) ):
        print('Cuidado, la lista padre contiene duplicados.'
             'Solo se utilizarán los valores únicos')
        
        parentList = list( set(parentList) )
        
        
    if len( childrenList ) != len( set(childrenList) ):
        print('Cuidado, la lista de hijos contiene duplicados.'
            'Solo se utilizarán los valores únicos')
        
        childrenList = list( set(childrenList) )
        
    # Un elemento de la lista parent no puede tener más
    # hijos únicos que el total de elementos en la lista children
    
    if offspring > len(childrenList):
        limit = len(childrenList)
        
        print('Se han indicado más hijos por padre que el número '
             'de elementos únicos en la lista de hijos. Se utiliza '
             'el total de elementos de la lista children.')
    else:
        limit = offspring
        
    # Creo una lista para almacenar los pares
    pairs = []

    for parent in parentList:
        
        # Hago esto para generar una copia de la lista
        # y poder modificarla sin afectar original
        candidates = list(childrenList)

        for n in range(limit):
            index = random.randint(0, len(candidates)-1 )
            child = candidates.pop(index)
            pairs.append( (parent, child) )
    
    print(f'Se han generado {len(pairs)} pares de elementos.')
    return pairs

        

<h2>1: NIVEL_ACCESO</h2>

In [146]:
def generateTableDf_NIVEL_ACCESO():
    data = {}

    # Generamos los datos
    niveles = ['A','B','C']

    print('Generando datos...')

    data['ID_NIVEL_ACCESO'] = [n+1 for n in range( len( niveles ) )]
    data['NIVEL'] = [ niveles[n] for n in range( len( niveles ) )]
    data['DESCRIPCION_NIVEL'] = [ toSpanish( fake.catch_phrase() ).title() for n in range( len(niveles) ) ]

    print('Listo')
    
    # Guardamos la data en un dataframe de pandas
    df = pd.DataFrame(data)
    
    return df

In [147]:
df_nivel_acceso = generateTableDf_NIVEL_ACCESO()
df_nivel_acceso.head()

Generando datos...
Listo


,ID_NIVEL_ACCESO,NIVEL,DESCRIPCION_NIVEL
0,1,A,Matriz Radical Asegurada
1,2,B,Groupware Dinamico Sincronizado
2,3,C,Producto Seguro Orientado Al Cliente


**Salida a CSV**

In [148]:
df_nivel_acceso.to_csv('csv/01-Nivel_acceso.csv', index=False, sep=';')

In [149]:
df_nivel_acceso = pd.read_csv('csv/01-Nivel_acceso.csv', sep=';')
df_nivel_acceso.head()

,ID_NIVEL_ACCESO,NIVEL,DESCRIPCION_NIVEL
0,1,A,Matriz Radical Asegurada
1,2,B,Groupware Dinamico Sincronizado
2,3,C,Producto Seguro Orientado Al Cliente


<h2 style="color:#777">2: EMPLEADO</h2>

**Funciones**

In [155]:
def generateFakeDNI(min=2, max=3):
    M = fake.random_int(min, max)
    rest = fake.numerify(text='#.###.###')
    return (f'{M}{rest}')

def generateFakeGender(nb_threshold = 0.05):
    choice = random.uniform(0, 1)
    
    if(choice < nb_threshold):
        return 'x'
    if(choice < 0.5):
        return 'M'
    else:
        return 'F'

def generateProfession():
    options = ["Tecnico Quimico", "Ingeniero Quimico", "Licenciado en Quimica", "Bioquimico", "Farmaceutico", "Biotecnologo"]
    return random.choice(options)

def generateGenderedName(gender):
    if gender == 'M':
        return fake.first_name_male()
    if gender == 'F':
        return fake.first_name_female()
    else:
        return fake.first_name_nonbinary()
    
def generateEmailFromName(first_name="john", last_name="doe"):
    provs = ['fakemail', 'truchomail', 'altomail','repiolamail']
    email = f'{first_name.lower()}.{last_name.lower()}{str(fake.random_int(min=100, max=999))}@{random.choice(provs)}.com'
    return email


**Generación de datos**

In [158]:
def generateTableDf_EMPLEADO(count=20):
    data = {}
    # Generamos la información básica de cada empleado

    print('Generando datos...')

    data['ID_EMPLEADO'] = [n+1 for n in range(count)]
    data['APELLIDO'] = [fake.last_name() for n in range(count)]
    data['DNI'] = [generateFakeDNI() for n in range(count)]
    data['GENERO'] = [generateFakeGender() for n in range(count)]
    data['TELEFONO_MOVIL'] = [fake.phone_number() for n in range(count)]
    data['FECHA_NACIMIENTO'] =[fake.date_between(start_date = "-65y", end_date = "-18y") for n in range(count)]
    data['TITULO_UNIVERSITARIO'] = [generateProfession() for n in range(count)]
    data['DIRECCION'] = [ removeAccentedChars(fake.address()) for n in range(count)]
    data['FECHA_CONTRATACION'] =[fake.date_between(start_date = "-10y", end_date = "-1y") for n in range(count)]
    data['ID_NIVEL_ACCESO'] = [fake.random_int(min=1, max=3) for n in range(count)]
    
    print('Listo')
    
    # Guardamos la data en un dataframe
    df = pd.DataFrame(data)
    
    # Sacamos los saltos de línea \n en la dirección:
    df['DIRECCION'] = df['DIRECCION'].apply(lambda x: x.replace(',', '-'))
    df['DIRECCION'] = df['DIRECCION'].apply(lambda x: x.replace('\n', '-'))

    # Generamos el nombre sabiendo el género
    df['NOMBRE'] = df.apply(lambda row: generateGenderedName(row['GENERO']), axis=1)

    # General el email sabiendo nombre y appellido
    df['EMAIL'] = df.apply(lambda row: generateEmailFromName(row['NOMBRE'], row['APELLIDO']), axis=1)

    return df

In [159]:
df_empleado = generateTableDf_EMPLEADO()
df_empleado.head()

Generando datos...
Listo


,ID_EMPLEADO,APELLIDO,DNI,GENERO,TELEFONO_MOVIL,FECHA_NACIMIENTO,TITULO_UNIVERSITARIO,DIRECCION,FECHA_CONTRATACION,ID_NIVEL_ACCESO,NOMBRE,EMAIL
0,1,Samper,29.929.713,F,+34 744639015,1995-06-05,Farmaceutico,Pasaje de Alejandro Marco 58 Puerta 4 -Valenci...,2015-12-10,1,Sonia,sonia.samper198@fakemail.com
1,2,Marcos,24.478.916,M,+34 870 905 797,1962-12-16,Ingeniero Quimico,Rambla Judith Ferrero 74-Murcia- 11650,2018-07-20,3,José Manuel,josé manuel.marcos599@altomail.com
2,3,Priego,33.050.456,M,+34642254375,1978-11-27,Biotecnologo,Pasadizo Patricio Blanes 1 Piso 1 -La Coruña- ...,2018-02-19,3,Lucho,lucho.priego624@fakemail.com
3,4,Ibañez,33.955.716,M,+34 622 12 23 23,1968-02-11,Ingeniero Quimico,Callejon Azahar Montenegro 241-Madrid- 44952,2013-03-07,3,Rómulo,rómulo.ibañez941@altomail.com
4,5,Salvà,24.420.998,M,+34889 88 73 87,1983-11-20,Licenciado en Quimica,Ronda Baldomero Cuesta 9 Apt. 29 -Guadalajara-...,2017-02-08,3,Aitor,aitor.salvà193@fakemail.com


**Salida a csv**

In [160]:
df_empleado.to_csv('csv/02-Empleado.csv', index=False, encoding='utf-8', sep=";")

In [161]:
df_empleado = pd.read_csv('csv/02-Empleado.csv', sep=";")
df_empleado.head()

,ID_EMPLEADO,APELLIDO,DNI,GENERO,TELEFONO_MOVIL,FECHA_NACIMIENTO,TITULO_UNIVERSITARIO,DIRECCION,FECHA_CONTRATACION,ID_NIVEL_ACCESO,NOMBRE,EMAIL
0,1,Samper,29.929.713,F,+34 744639015,1995-06-05,Farmaceutico,Pasaje de Alejandro Marco 58 Puerta 4 -Valenci...,2015-12-10,1,Sonia,sonia.samper198@fakemail.com
1,2,Marcos,24.478.916,M,+34 870 905 797,1962-12-16,Ingeniero Quimico,Rambla Judith Ferrero 74-Murcia- 11650,2018-07-20,3,José Manuel,josé manuel.marcos599@altomail.com
2,3,Priego,33.050.456,M,+34642254375,1978-11-27,Biotecnologo,Pasadizo Patricio Blanes 1 Piso 1 -La Coruña- ...,2018-02-19,3,Lucho,lucho.priego624@fakemail.com
3,4,Ibañez,33.955.716,M,+34 622 12 23 23,1968-02-11,Ingeniero Quimico,Callejon Azahar Montenegro 241-Madrid- 44952,2013-03-07,3,Rómulo,rómulo.ibañez941@altomail.com
4,5,Salvà,24.420.998,M,+34889 88 73 87,1983-11-20,Licenciado en Quimica,Ronda Baldomero Cuesta 9 Apt. 29 -Guadalajara-...,2017-02-08,3,Aitor,aitor.salvà193@fakemail.com


<h2 style="color:#777">3: PROYECTO</h2>

In [162]:
def generateProjectTitle():
    phrase = toSpanish(fake.catch_phrase())
    words = phrase.split()
    title =(f'{words[0]} {words[-1]}').title()
    return title

**Generación de datos**

In [163]:
def generateTableDf_PROYECTO(count=5):
    data = {}

    # Generamos la información básica de cada proyecto
    print('Generando datos...')

    data['ID_PROYECTO'] = [n+1 for n in range(count)]
    data['NOMBRE_PROYECTO'] = [generateProjectTitle() for n in range(count)]
    data['FECHA_CREACION'] =[fake.date_between(start_date = "-10y", end_date = "now") for n in range(count)]
    data['DESCRIPCION'] = [toSpanish(fake_en.paragraph(nb_sentences=3)) for n in range(count)]

    print('Listo')
    
    #Guardamos los datos en un dataframe
    df = pd.DataFrame(data)
    
    return df

In [164]:
# Guardamos la data en un dataframe
df_proyecto = generateTableDf_PROYECTO()
df_proyecto.head()

Generando datos...
Listo


,ID_PROYECTO,NOMBRE_PROYECTO,FECHA_CREACION,DESCRIPCION
0,1,Flexibilidad Generacion,2015-02-02,Los primeros medios tradicionales que bonitos ...
1,2,Sitio Configurable,2016-03-25,Comercializar a alguien cambiar la ley. Empuje...
2,3,Interfaz Direccional,2013-05-23,Mostrar servicio de estado verdadero de respue...
3,4,Producto Administrado,2019-09-18,Muchacho punto negro en realidad proposito pol...
4,5,Arquitectura Motivadora,2012-12-08,Resto PM probar ancho. Crimen por que color co...


In [165]:
df_proyecto.to_csv('csv/03-Proyecto.csv', index=False, encoding='utf-8', sep=';')

In [166]:
df_proyecto = pd.read_csv('csv/03-Proyecto.csv', sep=';')
df_proyecto.head()

,ID_PROYECTO,NOMBRE_PROYECTO,FECHA_CREACION,DESCRIPCION
0,1,Flexibilidad Generacion,2015-02-02,Los primeros medios tradicionales que bonitos ...
1,2,Sitio Configurable,2016-03-25,Comercializar a alguien cambiar la ley. Empuje...
2,3,Interfaz Direccional,2013-05-23,Mostrar servicio de estado verdadero de respue...
3,4,Producto Administrado,2019-09-18,Muchacho punto negro en realidad proposito pol...
4,5,Arquitectura Motivadora,2012-12-08,Resto PM probar ancho. Crimen por que color co...


<h2 style="color:#777">4: EXPERIMENTO</h2>

**Funciones**

In [167]:
def calculateEndTime(start, days=1):
    end = start + timedelta(days=days)
    return end
    

**Generación de datos**

In [168]:
def generateTableDf_EXPERIMENTO(fkeys, count=50):
    data = {}

    # Generamos la data principal

    print('Generando datos...')

    data['ID_EXPERIMENTO'] = [n+1 for n in range(count)]
    data['ID_PROYECTO'] = [ random.choice( fkeys['ID_PROYECTO'] ) for n in range(count) ]
    data['TITULO_EXPERIMENTO'] =[ toSpanish( fake.bs() ).title() for n in range(count) ]
    data['OBJETIVO'] = [toSpanish( fake_en.paragraph( nb_sentences=3 ) ) for n in range(count) ]
    data['DESCRIPCION_EXPERIMENTO'] = [ toSpanish( fake_en.paragraph( nb_sentences=3 ) ) for n in range(count)]
    data['CONCLUSION'] = [ toSpanish( fake_en.paragraph( nb_sentences=3 ) ) for n in range(count)]
    data['FECHA_INICIO'] =  [ fake.date_time_between( start_date = "-10y", end_date = "now" ) for n in range(count)]

    print('Listo')
    
    # Guardamos la data en un dataframe
    df = pd.DataFrame(data)
    
    # Se calcula la fecha de finalización como un día después del inicio
    df['FECHA_FINALIZACION'] = df['FECHA_INICIO'].apply(calculateEndTime)
    
    return df

In [169]:
fkeys={'ID_PROYECTO' : list(df_proyecto['ID_PROYECTO']) }

In [171]:
# df_experimento = generateTableDf_EXPERIMENTO(fkeys, count=50)
# df_experimento.head()

Generando datos...
Listo


,ID_EXPERIMENTO,ID_PROYECTO,TITULO_EXPERIMENTO,OBJETIVO,DESCRIPCION_EXPERIMENTO,CONCLUSION,FECHA_INICIO,FECHA_FINALIZACION
0,1,2,Aprovechar Los Mercados Electronicos De Proxim...,Establezca la tarjeta de region de escena de r...,Politica de consumo de taza de taza de taza de...,Use el proposito a veces. Intercambie la mitad...,2017-03-18 21:22:23,2017-03-19 21:22:23
1,2,5,Incentivar Los Servicios Web De Proxima Genera...,Lejos escribir orden hombre. Presion presupues...,Patron de director de decision de golpe de pod...,Evento de chicas a traves de una agradable sal...,2014-07-15 14:15:08,2014-07-16 14:15:08
2,3,2,Involucrar Esquemas De Codigo Abierto,Conozca los cuatro principales efectos de la s...,Puede sugerir cultura ciertamente. Una vez que...,Efecto de continuacion principal de Happy Cup....,2016-09-29 17:06:06,2016-09-30 17:06:06
3,4,3,Ofrecer Soluciones De Extremo A Extremo,Teoria luego mil dificil listo. Entrenador est...,Reflejo significativo del nombre del cientific...,Ataque padre misma semana. Bolsa niño seccion ...,2013-01-17 11:34:13,2013-01-18 11:34:13
4,5,5,Impulsar Aplicaciones De Valor Agregado,Resto americano discutir discutir chica todavi...,Ahora o desde entonces ofrecemos funda para ho...,Carrera pero comienzo familiar. Exito de sangr...,2013-12-17 00:33:52,2013-12-18 00:33:52


In [175]:
# Exportamos a csv
# df_experimento.to_csv('csv/04-Experimento.csv', index=False, encoding='utf-8', sep=';')

In [176]:
df_experimento = pd.read_csv('csv/04-Experimento.csv', sep=';')
df_experimento.head()

,ID_EXPERIMENTO,ID_PROYECTO,TITULO_EXPERIMENTO,OBJETIVO,DESCRIPCION_EXPERIMENTO,CONCLUSION,FECHA_INICIO,FECHA_FINALIZACION
0,1,2,Aprovechar Los Mercados Electronicos De Proxim...,Establezca la tarjeta de region de escena de r...,Politica de consumo de taza de taza de taza de...,Use el proposito a veces. Intercambie la mitad...,2017-03-18 21:22:23,2017-03-19 21:22:23
1,2,5,Incentivar Los Servicios Web De Proxima Genera...,Lejos escribir orden hombre. Presion presupues...,Patron de director de decision de golpe de pod...,Evento de chicas a traves de una agradable sal...,2014-07-15 14:15:08,2014-07-16 14:15:08
2,3,2,Involucrar Esquemas De Codigo Abierto,Conozca los cuatro principales efectos de la s...,Puede sugerir cultura ciertamente. Una vez que...,Efecto de continuacion principal de Happy Cup....,2016-09-29 17:06:06,2016-09-30 17:06:06
3,4,3,Ofrecer Soluciones De Extremo A Extremo,Teoria luego mil dificil listo. Entrenador est...,Reflejo significativo del nombre del cientific...,Ataque padre misma semana. Bolsa niño seccion ...,2013-01-17 11:34:13,2013-01-18 11:34:13
4,5,5,Impulsar Aplicaciones De Valor Agregado,Resto americano discutir discutir chica todavi...,Ahora o desde entonces ofrecemos funda para ho...,Carrera pero comienzo familiar. Exito de sangr...,2013-12-17 00:33:52,2013-12-18 00:33:52


<h2 style="color:#777">5: SUSTANCIA</h2>

**Funciones**

In [25]:
def generateSmiles(threshold=0.3):
    atoms = ['C','n','o','O','s','S','Cl','Br','I','F','(c)','(C)','(n)','(o)','(O)','(s)','(S)']
    length = fake.random_int(min=10, max=30)
    smiles_list = []
    for n in range(length):
        choice = random.uniform(0,1)
        if choice < threshold:
            smiles_list.append(random.choice(atoms))
        else:
            smiles_list.append('c')
    smiles = ''.join(smiles_list)
    return(smiles)


**Generación de datos**

In [26]:
def generateTableDf_SUSTANCIA(susts, fkeys, count=50):
    data = {}

    # Foreing Keys
    id_nivel_acceso = fkeys['ID_NIVEL_ACCESO']

    sustancias = [random.choice(susts) for n in range(count)]

    # Generamos la data

    print('Generando datos...')

    data['ID_SUSTANCIA'] = [n+1 for n in range(count)]
    data['ID_NIVEL_ACCESO'] = [random.choice(id_nivel_acceso) for n in range(count)]
    data['NOMBRE_COMUN'] = sustancias
    data['NRO_CAS'] = [fake.numerify(text='##-###-###') for n in range(count)]
    data['NOMBRE_IUPAC'] = [sust.lower().replace(' ','-') for sust in sustancias]
    data['SMILES'] = [generateSmiles() for n in range(count) ]
    data['PESO_MOLECULAR'] = [ float(fake.random_int(min=100, max=700) + random.uniform(0,1)) for n in range(count)  ]

    print('Listo')
    
    #Guardamos la data en un dataframe
    df = pd.DataFrame(data)
    
    return df

In [27]:
# Para los nombres de las sustancias uso una lista de productos
# obtenida de una distribuidora química, la que se encuentra en un
# archivo de texto, con una sustancia por línea. 
# Importamos las sustacias a una lista

filename = 'sustancias.txt'

with open(filename, encoding='utf-8') as file:
    susts = file.readlines()
    susts = [sust.rstrip() for sust in susts]
    
fkeys = {'ID_NIVEL_ACCESO': list(df_nivel_acceso['ID_NIVEL_ACCESO'])}

In [28]:
#Guardamos la data en un dataframe
# df_sustancia = generateTableDf_SUSTANCIA(susts, fkeys)
# df_sustancia.head()

In [177]:
# Exportamos a csv
#df_sustancia.to_csv('csv/05-Sustancia.csv', index=False, encoding='utf-8', sep=';')

In [178]:
df_sustancia = pd.read_csv('csv/05-Sustancia.csv', sep=';')
df_sustancia.head()

,ID_SUSTANCIA,ID_NIVEL_ACCESO,NOMBRE_COMUN,NRO_CAS,NOMBRE_IUPAC,SMILES,PESO_MOLECULAR
0,1,1,Alcohol Bencilico,28-125-394,alcohol-bencilico,cnIccc(O)cccc,446.161328
1,2,2,Bicarbonato de Sodio,02-270-431,bicarbonato-de-sodio,(c)ccccccssocccoccc(S)cnc(O)c,118.560761
2,3,1,Carbonato de Litio,13-450-361,carbonato-de-litio,ncc(o)ccccccccClcc(S)cc(n)ccc,228.671910
3,4,3,Pirofosfato Acido de Sodio,52-049-502,pirofosfato-acido-de-sodio,ccccnScc(S)c(o)cc(C)ccccccccc,636.926749
4,5,1,Acetona,09-535-890,acetona,nccc(o)c(C)cc(C)(o)cO,401.940669


<h2 style="color:#777">6: EQUIPAMIENTO</h2>

In [181]:
def generateTableDf_EQUIPAMIENTO(fkeys, count=20):
    data = {}

    id_nivel_acceso = fkeys['ID_NIVEL_ACCESO']
    
    equipos = [
        'Agitador Magnetico',
        'Agitador Mecanico',
        'Manto Calefactor',
        'pHmetro',
        'Balon 3 bocas 100ml',
        'Balon 3 bocas 250ml',
        'Balon 3 bocas 500ml',
        'Balon 3 bocas 1000ml',
        'Termometro electronico',
        'Refrigerante recto',
        'Viscosimetro',
        'Densimetro',
        'HPLC',
        'GC',
        'Espectrofotometro',
        'Balanza analitica',
        'Balanza granataria'    
    ]

    marcas = [
        'Garofalo S.A.',
        'Truch Instrumental',
        'Phalopa Inc.',
        'Juliet Pakard',
        'Sorny',
        'Sheneral Electric',
        'Supergood Chemical Solutions',
        'Cositorto Labware Enterprise'
    ]

    print('Generando Datos...')

    data['ID_EQUIPAMIENTO'] =  [ n+1 for n in range(count) ]
    data['ID_NIVEL_ACCESO'] = [ random.choice(id_nivel_acceso) for n in range(count) ]
    data['NOMBRE_EQUIPO'] = [ random.choice(equipos) for n in range(count) ]
    data['DESCRIPCION'] = [ 
        toSpanish( fake_en.paragraph(nb_sentences=2) ) 
        for n in range(count) 
    ]
    data['MARCA'] = [ random.choice(marcas) for n in range(count) ]
    data['MODELO'] = [ fake.numerify(text='#####-####') for n in range(count) ]

    print('listo')
    
    #Guardamos los datos en un dataframe
    df = pd.DataFrame(data)
    
    return df

In [182]:
fkeys = {'ID_NIVEL_ACCESO' : list( df_nivel_acceso['ID_NIVEL_ACCESO'] ) }

In [189]:
# df_equipamiento = generateTableDf_EQUIPAMIENTO(fkeys,count=20)
# df_equipamiento.head()

In [190]:
# Exportamos a csv
# df_equipamiento.to_csv('csv/06-Equipamiento.csv', index=False, encoding='utf-8', sep=';')

In [191]:
df_equipamiento = pd.read_csv('csv/06-Equipamiento.csv', sep=';')
df_equipamiento.head()

,ID_EQUIPAMIENTO,ID_NIVEL_ACCESO,NOMBRE_EQUIPO,DESCRIPCION,MARCA,MODELO
0,1,3,Balon 3 bocas 1000ml,Espera sobre el grado pero la oportunidad de o...,Sheneral Electric,28618-4328
1,2,3,Agitador Magnetico,"Radio puerta industria del sur. En cambio, nad...",Phalopa Inc.,90149-3060
2,3,3,Manto Calefactor,Sin embargo recuerdan la presion del mercado i...,Cositorto Labware Enterprise,91544-2647
3,4,3,Balon 3 bocas 250ml,Este elige como ser. Como celula vegetal a vec...,Sorny,14787-3999
4,5,1,Agitador Mecanico,Jugador presente desde hace mucho tiempo cultu...,Cositorto Labware Enterprise,01580-2253


<h2 style="color:#777">7: DOCUMENTO</h2>

In [36]:
def generateDocUrl(spanish=False):
    extensions = [
        '.docx',
        '.xls',
        '.txt',
        '.pdf',
        '.pptx'
    ]
    
    providers = [
        'madstorage.com',
        'amazondrive.',
        'gigaupload.com',
        'dropbasket.net'
    ]
    
    suburls = [
        '/usr/storage/',
        '/drive/files/',
        '/api/getfiles/'
    ]
    
    if spanish:
        filename = toSpanish(fake.catch_phrase()).replace(' ','-').lower()
        print('listo')
    else:
        filename = fake.catch_phrase().replace(' ','-').lower()
    
    e = random.choice(extensions)
    p = random.choice(providers)
    s = random.choice(suburls)
    
    url = f'https://{p}{s}{filename}{e}'
    return url
    


In [194]:
def generateTableDf_DOCUMENTO(fkeys, count=100):
    data = {}
    
    cats = ['Reporte','Presentacion','Informe','Protocolo','Resultado de analisis']
    
    print('Generating data...')
    
    data['ID_DOCUMENTO'] =  [ n+1 for n in range(count) ]
    data['ID_EXPERIMENTO'] = [ random.choice( fkeys['ID_EXPERIMENTO'] ) for n in range(count) ]
    data['CATEGORIA'] = [ random.choice( cats ) for n in range(count) ]
    data['NOMBRE_DOCUMENTO'] = [ fake.catch_phrase() for n in range(count) ]
    data['URL'] = [ generateDocUrl() for n in range(count) ]
    
    print('listo')
    
    df = pd.DataFrame(data)
    
    return df

In [195]:
fkeys = { 'ID_EXPERIMENTO': list( df_experimento['ID_EXPERIMENTO'] ) }

In [199]:
# df_documento = generateTableDf_DOCUMENTO(fkeys)
# df_documento.head()

In [200]:
# Exportamos a csv
# df_documento.to_csv('csv/07-Documento.csv', index=False, encoding='utf-8', sep=';')

In [201]:
df_documento = pd.read_csv('csv/07-Documento.csv', sep=';')
df_documento.head()

,ID_DOCUMENTO,ID_EXPERIMENTO,CATEGORIA,NOMBRE_DOCUMENTO,URL
0,1,22,Informe,Cross-group bandwidth-monitored utilization,https://gigaupload.com/drive/files/universal-d...
1,2,31,Resultado de analisis,Open-source uniform strategy,https://dropbasket.net/usr/storage/fully-confi...
2,3,8,Reporte,Inverse demand-driven approach,https://amazondrive./usr/storage/inverse-zero-...
3,4,46,Presentacion,Optimized interactive service-desk,https://dropbasket.net/api/getfiles/down-sized...
4,5,44,Informe,Down-sized foreground contingency,https://gigaupload.com/api/getfiles/front-line...


<h2 style="color:#777">8: MUESTRA</h2>

In [42]:
def generateTableDf_MUESTRA(fkeys, count=100):
    
    clases = ['Muestra de proceso', 'Materia Prima', 'Producto Elaborado']
    data = {}
    
    print('Generando datos...  ')
    
    data['ID_MUESTRA'] = [ n+1 for n in range(count) ]
    data['ID_EXPERIMENTO'] = [ random.choice( fkeys['ID_EXPERIMENTO'] ) for n in range(count) ]
    data['ID_EMPLEADO'] = [ random.choice( fkeys['ID_EMPLEADO'] ) for n in range(count) ]
    data['CODIGO_MUESTRA'] = [fake.numerify(text='###-###-###') for n in range(count) ]
    data['DESCRIPCION_MUESTRA'] = [fake_en.paragraph() for n in range(count) ]
    data['CLASE_MUESTRA'] =  [ random.choice( clases ) for n in range(count) ]
    data['FECHA_MUESTREO'] =  [ fake.date_time_between( start_date = "-10y", end_date = "now" ) for n in range(count) ]
    
    print('Listo...')
    
    df = pd.DataFrame(data)
    
    return df

In [43]:
fkeys = { 
    'ID_EXPERIMENTO': list( df_experimento['ID_EXPERIMENTO'] ),
    'ID_EMPLEADO': list( df_empleado['ID_EMPLEADO'] ),
        }

In [44]:
# df_muestra = generateTableDf_MUESTRA(fkeys)
# df_muestra.head()

In [202]:
# df_muestra.to_csv('csv/08-Muestra.csv', index=False, encoding='utf-8', sep=';')

In [203]:
df_muestra = pd.read_csv('csv/08-Muestra.csv', sep=';')
df_muestra.head()

,ID_MUESTRA,ID_EXPERIMENTO,ID_EMPLEADO,CODIGO_MUESTRA,DESCRIPCION_MUESTRA,CLASE_MUESTRA,FECHA_MUESTREO
0,1,31,15,624-389-800,Strategy weight probably ball let. Budget how ...,Materia Prima,2015-03-19 08:26:52
1,2,34,4,133-012-416,Personal region much education from. Able pret...,Producto Elaborado,2012-10-25 02:10:09
2,3,10,16,561-282-243,Against question not should. Section but opera...,Producto Elaborado,2017-08-27 16:38:28
3,4,2,2,501-755-469,Argue place fall security garden black none fa...,Producto Elaborado,2017-10-11 00:28:27
4,5,5,14,416-640-140,Commercial boy ten. A choice deal consumer feel.,Materia Prima,2014-01-10 14:26:22


<h2 style="color:#777">9: STOCK_SUSTANCIAS</h2>

In [227]:
def generateTableDf_STOCK_SUSTANCIAS( fkeys, count=200 ):
    data = {}
    
    ubicaciones = ['Deposito A', 'Deposito B', 'Bajomesada', 'Estante']
    unidades = ['g', 'kg', 'ml', 'l']
    marcas = ['WW Chemicals', 'Forza Pharmaceuticals', 'Quimica El Cholo', 'Simba Arlich', 'Merke' ]
    
    data['ID_STOCK_SUSTANCIAS'] = [ n+1 for n in range(count) ]
    data['ID_SUSTANCIA'] = [ random.choice( fkeys['ID_SUSTANCIA'] ) for n in range(count) ]
    data['UBICACION'] = [ random.choice(ubicaciones) for n in range(count) ]
    data['CANTIDAD'] = [ fake.random_int(min=50, max=500) for n in range(count) ]
    data['UNIDAD'] = [ random.choice(unidades) for n in range(count) ]
    data['PUREZA'] = [ float(fake.random_int(min=5, max=99) + random.uniform(0,1)) for n in range(count) ]
    data['MARCA'] = [ random.choice(marcas) for n in range(count) ]
    data['FECHA_INGRESO'] = [ fake.date_between( start_date="-10y", end_date="-1y" ) for n in range(count) ]
    
    df = pd.DataFrame(data)
    
        # Se calcula la fecha de finalización como un día después del inicio
    df['FECHA_CADUCIDAD'] = df['FECHA_INGRESO'].apply( lambda x : calculateEndTime(x, days=730) )
    
    return df

In [228]:
fkeys = {
    'ID_SUSTANCIA': list( df_sustancia['ID_SUSTANCIA'] )
}

In [229]:
df_stock_sustancias = generateTableDf_STOCK_SUSTANCIAS(fkeys)
df_stock_sustancias.head()

,ID_STOCK_SUSTANCIAS,ID_SUSTANCIA,UBICACION,CANTIDAD,UNIDAD,PUREZA,MARCA,FECHA_INGRESO,FECHA_CADUCIDAD
0,1,5,Bajomesada,235,g,63.636732,Merke,2020-10-07,2022-10-07
1,2,26,Deposito B,53,kg,8.434768,Quimica El Cholo,2014-04-25,2016-04-24
2,3,42,Estante,385,g,50.400149,Merke,2019-08-29,2021-08-28
3,4,38,Deposito A,286,ml,43.568929,Forza Pharmaceuticals,2017-03-09,2019-03-09
4,5,34,Bajomesada,242,l,23.498604,Simba Arlich,2014-08-26,2016-08-25


In [230]:
# Exportamos a csv
df_stock_sustancias.to_csv('csv/09-Stock_sustancias.csv', index=False, encoding='utf-8', sep=';')

In [231]:
df_stock_sustancias = pd.read_csv('csv/09-Stock_sustancias.csv', sep=';')
df_stock_sustancias.head()

,ID_STOCK_SUSTANCIAS,ID_SUSTANCIA,UBICACION,CANTIDAD,UNIDAD,PUREZA,MARCA,FECHA_INGRESO,FECHA_CADUCIDAD
0,1,5,Bajomesada,235,g,63.636732,Merke,2020-10-07,2022-10-07
1,2,26,Deposito B,53,kg,8.434768,Quimica El Cholo,2014-04-25,2016-04-24
2,3,42,Estante,385,g,50.400149,Merke,2019-08-29,2021-08-28
3,4,38,Deposito A,286,ml,43.568929,Forza Pharmaceuticals,2017-03-09,2019-03-09
4,5,34,Bajomesada,242,l,23.498604,Simba Arlich,2014-08-26,2016-08-25


<h2 style="color:#777">10: STOCK_EQUIPOS</h2>

In [244]:
# Encierro la lógica de la generación de cada tabla
# en una función que devuelve un dataframe.
# El template de la función es:

def generateTableDf_STOCK_EQUIPOS(fkeys, count=20):
    data = {}
    
    ubicaciones = ['Deposito A', 'Deposito B', 'Bajomesada', 'Estante']
    
    data['ID_STOCK_EQUIPOS'] = [ n+1 for n in range(count) ]
    data['ID_EQUIPAMIENTO'] = [ random.choice( fkeys['ID_EQUIPAMIENTO'] ) for n in range(count) ]
    data['UBICACION'] = [ random.choice(ubicaciones) for n in range(count) ]
    data['CANTIDAD'] = [ fake.random_int(min=1, max=10) for n in range(count) ]
    data['FECHA_INGRESO'] = [ fake.date_between( start_date="-10y", end_date="now" ) for n in range(count) ]
    
    df = pd.DataFrame(data)
    
    return df

In [245]:
fkeys = {
    'ID_EQUIPAMIENTO': list( df_equipamiento['ID_EQUIPAMIENTO'] )
}

In [246]:
df_stock_equipos = generateTableDf_STOCK_EQUIPOS(fkeys)
df_stock_equipos.head()

,ID_STOCK_EQUIPOS,ID_EQUIPAMIENTO,UBICACION,CANTIDAD,FECHA_INGRESO
0,1,20,Deposito B,8,2017-03-26
1,2,8,Deposito A,10,2019-12-22
2,3,9,Deposito B,6,2020-06-05
3,4,13,Deposito A,6,2019-04-27
4,5,15,Deposito B,7,2014-04-15


In [247]:
# Exportamos a csv
df_stock_equipos.to_csv('csv/10-Stock_equipos.csv', index=False, encoding='utf-8', sep=';')

In [248]:
df_stock_equipos = pd.read_csv('csv/10-Stock_equipos.csv', sep=';')
df_stock_equipos.head()

,ID_STOCK_EQUIPOS,ID_EQUIPAMIENTO,UBICACION,CANTIDAD,FECHA_INGRESO
0,1,20,Deposito B,8,2017-03-26
1,2,8,Deposito A,10,2019-12-22
2,3,9,Deposito B,6,2020-06-05
3,4,13,Deposito A,6,2019-04-27
4,5,15,Deposito B,7,2014-04-15


<h2 style="color:#777">11: PROYECTO_EMPLEADO</h2>

In [57]:
def generateTableDf_PROYECTO_EMPLEADO(fkeys, count=20, offspring=5):
    
    # offpring: nro de empleados por proyecto
    # count: cantidad de pares empleado-proyecto a retornar
    data = {}
    
    roles = [ 'Creador', 'Miembro', 'Invitado' ]
    
    # Generamos pares proyecto - empleado
    # Por cada proyecto se asocian N empleados al azar, siendo N el valor indicado en offspring
    # Los mismos empleados trabajan en muchos proyectos, pero no se repiten dentro de cada proyecto.
    
    pairs = generateParentChildPairs( fkeys['ID_PROYECTO'], fkeys['ID_EMPLEADO'], offspring )
    
    # Muestreamos sin reposición:
    pairs = random.sample(pairs, count) 
    
    data['ID_PROYECTO'] = [ pair[0] for pair in pairs ]
    data['ID_EMPLEADO'] = [ pair[1] for pair in pairs ]
    data['ROL'] = [ random.choice(roles) for n in range(count)  ]
    df = pd.DataFrame(data)
    
    return df
    

In [58]:
fkeys = {
    'ID_PROYECTO': list( df_proyecto['ID_PROYECTO'] ),
    'ID_EMPLEADO': list( df_empleado['ID_EMPLEADO'] )
}

In [59]:
# df_proyecto_empleado = generateTableDf_PROYECTO_EMPLEADO(fkeys, count=50, offspring=10)
# df_proyecto_empleado.head()

In [252]:
# Exportamos a csv
df_proyecto_empleado.to_csv('csv/11-Proyecto_empleado.csv', index=False, encoding='utf-8', sep=';')

In [253]:
df_proyecto_empleado = pd.read_csv('csv/11-Proyecto_empleado.csv', sep=';')
df_proyecto_empleado.head()

,ID_PROYECTO,ID_EMPLEADO,ROL
0,1,7,Ejecutor
1,2,4,Revisor
2,3,11,Suepervisor
3,4,11,Suepervisor
4,3,3,Suepervisor


<h2 style="color:#777">12: EXPERIMENTO_EMPLEADO</h2>

In [62]:
def generateTableDf_EXPERIMENTO_EMPLEADO(fkeys, count=10, offspring=3):
    # offpring: nro de empleados por proyecto
    # count: cantidad de pares empleado-experimento a retornar
    data = {}
    
    roles = [ 'Supervisor', 'Ejecutor', 'Responsable', 'Observador' ]
    
    # Generamos pares experimento - empleado
    # Por cada experimento se asocian N empleados al azar, siendo N el valor indicado en offspring
    # Los mismos empleados trabajan en muchos experimentos, pero no se repiten dentro de cada experimenmto.
    
    # Nota: dado que no está tomando en cuenta lo que pasa en la tabla proyecto_empleado,
    # se van a generar pares experimento_empleado donde el empleado no forma parte del proyecto
    # al cual corresponde el experimento. Dicho de otra forma,no existen restricciones en cuanto
    # a la pertenencia a un proyecto, en los experimentos pueden trabajar
    # 'invitados' que vienen de otros proyectos.
    
    pairs = generateParentChildPairs( fkeys['ID_EXPERIMENTO'], fkeys['ID_EMPLEADO'], offspring )
    
    # Muestreamos sin reposición:
    pairs = random.sample(pairs, count) 
    
    data['ID_EXPERIMENTO'] = [ pair[0] for pair in pairs ]
    data['ID_EMPLEADO'] = [ pair[1] for pair in pairs ]
    data['ROL'] = [ random.choice(roles) for n in range(count)  ]
    df = pd.DataFrame(data)
    
    return df

In [63]:
fkeys = {
    'ID_EXPERIMENTO': list( df_experimento['ID_EXPERIMENTO'] ),
    'ID_EMPLEADO': list( df_empleado['ID_EMPLEADO'] )
}

In [64]:
# df_experimento_empleado = generateTableDf_EXPERIMENTO_EMPLEADO(fkeys, count=100)
# df_experimento_empleado.head()

In [254]:
# Exportamos a csv
df_experimento_empleado.to_csv('csv/12-Experimento_empleado.csv', index=False, encoding='utf-8', sep=';')

In [255]:
df_experimento_empleado = pd.read_csv('csv/12-Experimento_empleado.csv', sep=';')
df_experimento_empleado.head()

,ID_EXPERIMENTO,ID_EMPLEADO,ROL
0,15,10,Supervisor
1,11,8,Observador
2,20,5,Responsable
3,31,2,Ejecutor
4,39,13,Responsable


<h2 style="color:#777">13: EXPERIMENTO_EQUIPOS</h2>

In [88]:
def generateTableDf_EXPERIMENTO_EQUIPOS(fkeys, count=10, offspring=3):
    # offpring: nro de empleados por proyecto
    # count: cantidad de pares empleado-experimento a retornar
    data = {}
    
    # Generamos pares experimento - equipos
    # Por cada experimento se asocian N equipos al azar, siendo N el valor indicado en offspring
    # Los mismos equipos se usan en muchos experimentos, pero no se repiten dentro de cada experimenmto.
    
    # cuidado con la cantidad: dado que no toma en cuenta lo que ocurre en la tabla stock, puede ser
    # que la cantidad asignada al azar supere el stock
        
    pairs = generateParentChildPairs( fkeys['ID_EXPERIMENTO'], fkeys['ID_STOCK_EQUIPOS'], offspring )
    
    # Muestreamos sin reposición:
    pairs = random.sample(pairs, count) 
    
    data['ID_EXPERIMENTO'] = [ pair[0] for pair in pairs ]
    data['ID_STOCK_EQUIPOS'] = [ pair[1] for pair in pairs ]
    data['CANTIDAD'] = [ fake.random_int(min=1, max=2) for n in range(count)  ]
    df = pd.DataFrame(data)
    
    return df

In [89]:
fkeys = {
    'ID_EXPERIMENTO': list( df_experimento['ID_EXPERIMENTO'] ),
    'ID_STOCK_EQUIPOS': list( df_stock_equipos['ID_STOCK_EQUIPOS'] )
}

In [90]:
# df_experimento_equipamiento = generateTableDf_EXPERIMENTO_EQUIPOS(fkeys, count=10, offspring=3)
# df_experimento_equipamiento.head()

In [256]:
# Exportamos a csv:
df_experimento_equipamiento.to_csv('csv/13-Experimento_equipos.csv', index=False, encoding='utf-8', sep=';')

In [257]:
# Exportamos a csv:
df_experimento_equipamiento = pd.read_csv('csv/13-Experimento_equipos.csv', sep=';')
df_experimento_equipamiento.head()

,ID_EXPERIMENTO,ID_STOCK_EQUIPOS,CANTIDAD
0,38,16,2
1,30,20,1
2,27,11,2
3,38,19,1
4,48,18,1


<h2 style="color:#777">14: EXPERIMENTO_SUSTANCIAS</h2>

In [72]:
def generateTableDf_EXPERIMENTO_SUSTANCIAS(fkeys, count=100):
    data = {}
    
    pairs = generateParentChildPairs( fkeys['ID_EXPERIMENTO'], fkeys['ID_SUSTANCIA'], offspring=5)
    
    # Muestreamos sin reposición:
    pairs = random.sample(pairs, count)
    
    
    data['ID_EXPERIMENTO'] = [ pair[0] for pair in pairs ]
    data['ID_SUSTANCIA'] = [ pair[1] for pair in pairs ]
    data['CANTIDAD'] = [ float(fake.random_int(min=100, max=700) + random.uniform(0,1)) for n in range(count)  ]
    df = pd.DataFrame(data)
    
    return df

In [73]:
fkeys = {
    'ID_EXPERIMENTO': list( df_experimento['ID_EXPERIMENTO'] ),
    'ID_SUSTANCIA': list( df_sustancia['ID_SUSTANCIA'] )
}

In [74]:
# df_experimento_sustancia = generateTableDf_EXPERIMENTO_SUSTANCIAS(fkeys)
# df_experimento_sustancia.head()

In [258]:
# Exportamos a csv
df_experimento_sustancia.to_csv('csv/14-Experimento_sustancias.csv', index=False, encoding='utf-8', sep=';')

In [259]:
df_experimento_sustancia = pd.read_csv('csv/14-Experimento_sustancias.csv', sep=';')
df_experimento_sustancia.head()

,ID_EXPERIMENTO,ID_SUSTANCIA,CANTIDAD
0,35,11,388.573128
1,10,36,340.105112
2,19,50,433.197640
3,23,30,261.429803
4,42,43,571.882002


<h2 style="color:#777">15: MOVIMIENTO_SUSTANCIAS</h2>

In [77]:
def generateOperationDate(reference_date):
    if reference_date > datetime.today():
        return reference_date

    days = (datetime.today() - reference_date).days

    if days == 0:
        return reference_date

    random_delta = fake.random_int(min=0, max=days)

    date = calculateEndTime(reference_date, days= random_delta)

    return date
    
    
    

In [78]:
def generateTableDf_MOVIMIENTO_SUSTANCIAS(fkeys, count=10):
    data = {}
    
    operaciones = ['Ingreso', 'Consumo']
    
    data['ID_MOVIMIENTO_SUSTANCIAS'] = [ n+1 for n in range(count) ]
    data['ID_STOCK_SUSTANCIAS'] = [ random.choice( fkeys['ID_STOCK_SUSTANCIAS'] ) for n in range(count) ]
    data['ID_EMPLEADO'] = [ random.choice( fkeys['ID_EMPLEADO'] ) for n in range(count) ]
    data['OPERACION'] = [ random.choice( operaciones ) for n in range(count) ]
    data['CANTIDAD'] = [ float(fake.random_int(min=100, max=500) + random.uniform(0,1)) for n in range(count) ]
    
   
    df = pd.DataFrame(data)
    
    # Columna auxiliar con las fechas de referencia
    df['FECHA'] = df['ID_STOCK_SUSTANCIAS'].apply(lambda x: df_stock_sustancias['FECHA_INGRESO'][x])

    # Calculo la fecha de operación entre algún momento y ahora
    df['FECHA_OPERACION'] = df['FECHA'].apply(lambda x: generateOperationDate(datetime.strptime(x, '%Y-%m-%d')))
    
    df = df.drop('FECHA', axis=1)
    
    return df

In [79]:
fkeys = {
    'ID_STOCK_SUSTANCIAS': list( df_stock_sustancias['ID_STOCK_SUSTANCIAS'] ),
    'ID_EMPLEADO': list( df_empleado['ID_EMPLEADO'] )
}

In [80]:
# df_movimiento_sustancias = generateTableDf_MOVIMIENTO_SUSTANCIAS(fkeys, count=50)

In [260]:
# Exportamos a csv
df_movimiento_sustancias.to_csv('csv/15-Movimiento_sustancias.csv', index=False, encoding='utf-8', sep=';')

In [261]:
df_movimiento_sustancias = df_movimiento_sustancias = pd.read_csv('csv/15-Movimiento_sustancias.csv', sep=';')
df_movimiento_sustancias.head()

,ID_MOVIMIENTO_SUSTANCIAS,ID_STOCK_SUSTANCIAS,ID_EMPLEADO,OPERACION,CANTIDAD,FECHA_OPERACION
0,1,35,18,Consumo,174.190337,2015-11-29
1,2,172,3,Ingreso,168.686407,2018-02-08
2,3,13,5,Ingreso,394.131921,2017-05-04
3,4,144,20,Ingreso,329.104693,2019-06-20
4,5,66,14,Consumo,492.035348,2015-01-08


<h2 style="color:#777">16: MOVIMIENTO_EQUIPOS</h2>
<a id='tabla_16'></a>

In [83]:
def generateTableDf_MOVIMIENTO_EQUIPOS(fkeys, count=10):
    data = {}
    
    operaciones = ['Alta', 'Baja']
    
    data['ID_MOVIMIENTO_EQUIPOS'] = [ n+1 for n in range(count) ]
    data['ID_STOCK_EQUIPOS'] = [ random.choice( fkeys['ID_STOCK_EQUIPOS'] ) for n in range(count) ]
    data['ID_EMPLEADO'] = [ random.choice( fkeys['ID_EMPLEADO'] ) for n in range(count) ]
    data['OPERACION'] = [ random.choice( operaciones ) for n in range(count) ]
    data['CANTIDAD'] = [ fake.random_int(min=1, max=3) for n in range(count) ]
    
   
    df = pd.DataFrame(data)
    
    # Columna auxiliar con las fechas de referencia
    df['FECHA'] = df['ID_STOCK_EQUIPOS'].apply(lambda x: df_stock_equipos['FECHA_INGRESO'][x])

    # Calculo la fecha de operación entre algún momento y ahora
    df['FECHA_OPERACION'] = df['FECHA'].apply(lambda x: generateOperationDate(datetime.strptime(x, '%Y-%m-%d')))
    
    df = df.drop('FECHA', axis=1)
    
    return df

In [84]:
fkeys = {
    'ID_STOCK_EQUIPOS': list( df_stock_equipos['ID_STOCK_EQUIPOS'] ),
    'ID_EMPLEADO': list( df_empleado['ID_EMPLEADO'] )
}

In [85]:
# df_movimiento_equipos = generateTableDf_MOVIMIENTO_EQUIPOS(fkeys, count=50)

In [86]:
# Exportamos a csv
df_movimiento_equipos.to_csv('csv/16-Movimiento_equipos.csv', index=False , encoding='utf-8', sep=';')

In [87]:
df_movimiento_equipos = pd.read_csv('csv/16-Movimiento_equipos.csv')
df_movimiento_equipos.head()

,ID_MOVIMIENTO_EQUIPOS,ID_STOCK_EQUIPOS,ID_EMPLEADO,OPERACION,CANTIDAD,FECHA_OPERACION
0,1,14,8,Baja,1,2021-09-16
1,2,2,18,Baja,2,2020-05-29
2,3,3,10,Alta,3,2021-08-20
3,4,15,3,Alta,2,2020-08-18
4,5,2,9,Baja,3,2020-10-03
